In [5]:
import pandas as pd
import numpy as np

In [11]:
from sklearn.preprocessing import MinMaxScaler

AttributeError: module 'scipy.sparse' has no attribute 'linalg'

In [ ]:
df = pd.read_csv("March_2024_AF2_01.txt", sep=r"\s+", header=None)
df.columns = ['Protein_number', 'res_number', 'PB1', 'PB2', 'AA1', 'AA2', 'S2_1', 'S2_2', 'expected_frequency', 'plDDT', 'RSA1', 'RSA2']
df

In [ ]:
numerical_cols = ['expected_frequency', 'plDDT', 'RSA1', 'RSA2']
categorical_cols = ['PB1', 'PB2', 'AA1', 'AA2', 'S2_1', 'S2_2']

###  Normalize numerical columns

In [ ]:
scaler = MinMaxScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [ ]:
for col in categorical_cols:
    df[col] = df[col].astype('category').cat.codes

In [ ]:
# Convert to NumPy array (K-Prototypes requires this format)
data = df[numerical_cols + categorical_cols].to_numpy()

## K-Prototypes clustering

In [7]:
from kmodes.kprototypes import KPrototypes

AttributeError: module 'scipy.sparse' has no attribute 'linalg'

In [ ]:
ks = range(5, 50, 5)

In [ ]:
kproto = KPrototypes(n_clusters=5, init='Huang', random_state=42)
clusters = kproto.fit_predict(data, categorical=list(range(len(numerical_cols), data.shape[1])))